## Train, tune, and deploy a custom ML model using Financial Transactions Fraud Detection Algorithm from AWS Marketplace 


This solution is a deep learning based algorithm capable of training on transactional data, to identify spurious transactions.



This sample notebook shows you how to train a custom ML model using Financial Transactions Fraud Detection from AWS Marketplace.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to For Seller to update: Financial Transactions Fraud Detection. 

#### Contents:
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure and visualize train and test dataset](#B.-Configure-and-visualize-train-and-test-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Train a machine learning model](#3:-Train-a-machine-learning-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Train a model](#3.2-Train-a-model)
1. [Deploy model and verify results](#4:-Deploy-model-and-verify-results)
    1. [Deploy trained model](#A.-Deploy-trained-model)
    1. [Create input payload](#B.-Create-input-payload)
    1. [Perform real-time inference](#C.-Perform-real-time-inference)
    1. [Visualize output](#D.-Visualize-output)
    1. [Calculate relevant metrics](#E.-Calculate-relevant-metrics)
    1. [Delete the endpoint](#F.-Delete-the-endpoint)
1. [Tune your model! (optional)](#5:-Tune-your-model!-(optional))
	1. [Tuning Guidelines](#A.-Tuning-Guidelines)
	1. [Define Tuning configuration](#B.-Define-Tuning-configuration)
	1. [Run a model tuning job](#C.-Run-a-model-tuning-job)
1. [Perform Batch inference](#6.-Perform-Batch-inference)
1. [Clean-up](#7.-Clean-up)
	1. [Delete the model](#A.-Delete-the-model)
	1. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))


#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page Financial Transactions Fraud Detection
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [22]:
algo_arn='arn:aws:sagemaker:us-east-2:786796469737:algorithm/fraud-fin-tensor-1'

### 2. Prepare dataset

In [1]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import urllib.request
import numpy as np

#### A. Dataset format expected by the algorithm

The algorithm requires data in the format as decribed for best results 

1. Input data should be in binarized format to train and learn the patterns 

1. Build the binary variables judiciously and develop binary encoded columns, rather generating one-hot binary variables 

1. Keep the ratio of fraudulent data to non-fraudulent more then 1:1000 

1. Try to incorporate as much patterns from non-fraudulent data as possible to increase out of sample accuracy 

#### B. Configure and visualize train and test dataset

In [3]:
training_dataset='train.csv'

In [4]:
test_dataset='test.csv'

In [5]:
import pandas as pd
df = pd.read_csv(training_dataset)
df.head()

,month_0,month_1,month_2,month_3,month_4,date_0,date_1,date_2,date_3,date_4,...,amt___3,amt___4,amt___5,amt___6,amt___7,amt___8,swap_diff__0,swap_diff__1,swap_diff__2,is_fraud
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
2,0,0,0,1,1,0,0,0,0,1,...,0,0,0,0,1,1,0,0,1,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0,1,0,1,0,0,0,1,0,...,0,0,0,1,0,1,0,1,1,0


#### C. Upload datasets to Amazon S3

In [6]:
sagemaker_session = sage.Session()
bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

In [7]:
training_data=sagemaker_session.upload_data(training_dataset, bucket=bucket, key_prefix='financial_fraud')
test_data=sagemaker_session.upload_data(test_dataset, bucket=bucket, key_prefix='financial_fraud')

## 3: Train a machine learning model

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to train a machine learning model. 

### 3.1 Set up environment

In [8]:
role = get_execution_role()


In [9]:
output_location = 's3://{}/financial_fraud/{}'.format(bucket, 'output')

### 3.2 Train a model

You can also find more information about hyperparametes in **Hyperparameters** section of Financial Transactions Fraud Detection

In [11]:
#Define hyperparameters
hyperparameters={"epoches":100,"batch_size":10}

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [23]:
#Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="financial-fraud-training",
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m5.large',
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type='ml.m5.large'
)
#Run the training job.
estimator.fit({"training": training_data,"test":test_data})

2021-05-10 17:38:51 Starting - Starting the training job...
2021-05-10 17:39:15 Starting - Launching requested ML instancesProfilerReport-1620668331: InProgress
......
2021-05-10 17:40:15 Starting - Preparing the instances for training...
2021-05-10 17:40:52 Downloading - Downloading input data...
2021-05-10 17:41:15 Training - Downloading the training image...
2021-05-10 17:41:51 Training - Training image download completed. Training in progress..2021-05-10 17:41:52.358827: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-05-10 17:41:52.358868: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-05-10 17:41:56.089488: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-10 17:41:56.0918

See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

### 4: Deploy model and verify results

Now you can deploy the model for performing real-time inference.

In [17]:
model_name='financial_fraud_inference'

content_type='text/csv'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Deploy trained model

In [32]:
from sagemaker.predictor import csv_serializer
predictor = estimator.deploy(1, real_time_inference_instance_type, serializer=csv_serializer)

..........
-------------!

Once endpoint is created, you can perform real-time inference.

#### B. Create input payload

In [69]:
df = pd.read_csv("infrence.csv")

In [82]:
print(predictor.predict(df.values))

"1"


#### C. Perform real-time inference

In [43]:
file_name = "infrence.csv"
output_file_name = "result.txt"

In [44]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $predictor.endpoint \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    $output_file_name

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Visualize output

In [45]:
with open("result.txt","r") as file:
    print(file.read())

"1"


#### F. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. you can terminate the same to avoid being charged.

In [83]:
predictor.delete_endpoint(delete_endpoint_config=True)

Since this is an experiment, you do not need to run a hyperparameter tuning job. However, if you would like to see how to tune a model trained using a third-party algorithm with Amazon SageMaker's hyperparameter tuning functionality, you can run the optional tuning step.

### 5. Perform Batch inference

In this section, you will perform batch inference using multiple input payloads together.

In [84]:
#upload the batch-transform job input files to S3
transform_input_folder = "infrence.csv"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/financial_fraud_inference/infrence.csv


In [86]:
#Run the batch-transform job
transformer = estimator.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

..........
........................
. * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [10/May/2021 18:33:00] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [10/May/2021 18:33:00] "#033GET /execution-parameters HTTP/1.1#033" 404 -
169.254.255.130 - - [10/May/2021 18:33:01] "#033POST /invocations HTTP/1.1#033" 200 -
169.254.255.130 - - [10/May/2021 18:33:00] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [10/May/2021 18:33:00] "#033GET /execution-parameters HTTP/1.1#033" 404 -
169.254.255.130 - - [10/May/2021 18:33:01] "#033POST /invocations HTTP/1.1#033" 200 -
2021-05-10T18:33:00.830:[sagemaker logs]: MaxConcurrentTransforms=1, 

In [87]:
#output is available on following path
transformer.output_path

's3://sagemaker-us-east-2-786796469737/financial-fraud-training-2021-05-10-18-29-02-689'

### 7. Clean-up

#### A. Delete the model

In [91]:
estimator.delete_endpoint()

The function delete_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

